# データの読み込み

In [1]:
from PIL import Image
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
import torch.functional as F
import torchvision.transforms as transforms
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import skorch
import pandas as pd
import sklearn
import csv
import os

In [2]:
l_classlist = ['4gokan', '5gokan', '5gokan-denkisogokan', '5gokan-mediacenter'
                    '8gokan', 'denkisogokan', 'lab', 'lab-lounge', 'lounge']
s_classlist = ['4gokan_inside', '4gokan_outside', '5gokan-denkisogokan', '5gokan-mediacenter',
'5gokan_1F', '5gokan_2F', '5gokan_3F', '5gokan_ent_east', '5gokan_lounge',
'5gokan_out_east', '5gokan_parking', '5gokan_smoking', '5gokan_stairs_cnt',
'5gokan_stairs_west', '8gokan_1F', '8gokan_ent_north', '8gokan_ent_south',
'denkisogokan_2F', 'denkisogokan_3F', 'denkisogokan_4F', 'denkisogokan_elevator',
'denkisogokan_lounge', 'denkisogokan_stairs', 'lab-lounge', 'lab_bs_cnt',
'lab_corner', 'lab_desk', 'lab_desk_table', 'lab_ent', 'lab_printer',
'lab_table', 'lab_wb_cnt', 'lab_wb_ent', 'lounge']
num_l_class =len(l_classlist)
num_s_class = len(s_classlist)
print(num_s_class)

34


In [3]:
DATA_FOLDER = '../data/'
csv_path = os.path.join(DATA_FOLDER, 'frames_data.csv')
datalist = pd.read_csv(os.path.join(DATA_FOLDER, 'frames_data.csv'), names=["img_path", "l_class", 's_class','timestamp'])
datalist.head()

,img_path,l_class,s_class,timestamp
0,../data/insta_frames/5gokan-denkisogokan/5goka...,5gokan-denkisogokan,5gokan-denkisogokan,2019/10/23 18:50:43
1,../data/insta_frames/5gokan-denkisogokan/5goka...,5gokan-denkisogokan,5gokan-denkisogokan,2019/10/23 18:50:43
2,../data/insta_frames/5gokan-denkisogokan/5goka...,5gokan-denkisogokan,5gokan-denkisogokan,2019/10/23 18:50:43
3,../data/insta_frames/5gokan-denkisogokan/5goka...,5gokan-denkisogokan,5gokan-denkisogokan,2019/10/23 18:50:43
4,../data/insta_frames/5gokan-denkisogokan/5goka...,5gokan-denkisogokan,5gokan-denkisogokan,2019/10/23 18:50:43


In [4]:
dfl = datalist.drop(['s_class','timestamp'], axis=1)
dfl.head()

,img_path,l_class
0,../data/insta_frames/5gokan-denkisogokan/5goka...,5gokan-denkisogokan
1,../data/insta_frames/5gokan-denkisogokan/5goka...,5gokan-denkisogokan
2,../data/insta_frames/5gokan-denkisogokan/5goka...,5gokan-denkisogokan
3,../data/insta_frames/5gokan-denkisogokan/5goka...,5gokan-denkisogokan
4,../data/insta_frames/5gokan-denkisogokan/5goka...,5gokan-denkisogokan


In [5]:
dfl.groupby('l_class').count()

,img_path
l_class,
4gokan,308
5gokan,1328
5gokan-denkisogokan,361
5gokan-mediacenter,127
8gokan,266
denkisogokan,523
lab,1953
lab-lounge,63
lounge,247


In [6]:
dfs = datalist.drop(['l_class', 'timestamp'], axis=1)
dfs.groupby('s_class').count()

,img_path
s_class,
4gokan_inside,60
4gokan_outside,248
5gokan-denkisogokan,361
5gokan-mediacenter,127
5gokan_1F,65
5gokan_2F,96
5gokan_3F,32
5gokan_ent_east,135
5gokan_lounge,361


In [7]:
dfs.img_path.count()

5176

In [8]:
labs = dfs[dfs['s_class'].str.startswith('lab')]
num_labs_class = 10
labs.s_class.unique()


array(['lab-lounge', 'lab_ent', 'lab_wb_ent', 'lab_bs_cnt',
       'lab_desk_table', 'lab_desk', 'lab_corner', 'lab_printer',
       'lab_wb_cnt', 'lab_table'], dtype=object)

In [9]:
cam_test_img_path =[
    "../data/insta_frames/4gokan/4gokan_inside/4gokan_inside_0_img_000240.png",
    "../data/insta_frames/4gokan/4gokan_outside/4gokan_outside_0_img_000180.png",
    "../data/insta_frames/5gokan-denkisogokan/5gokan-denkisogokan_1_img_000010.png",
    "../data/insta_frames/5gokan-mediacenter/5gokan-mediacenter_0_img_000030.png",
    "../data/insta_frames/5gokan/5gokan_1F/5gokan_1F_0_img_000070.png",
    "../data/insta_frames/5gokan/5gokan_2F/5gokan_2F_2_img_000030.png",
    "../data/insta_frames/5gokan/5gokan_3F/5gokan_3F_0_img_000040.png",
    "../data/insta_frames/5gokan/5gokan_ent_east/5gokan_ent_east_2_img_000000.png",
    "../data/insta_frames/5gokan/5gokan_lounge/5gokan_lounge_2_img_000060.png",
    "../data/insta_frames/5gokan/5gokan_out_east/5gokan_out_east_0_img_000270.png",
    "../data/insta_frames/5gokan/5gokan_parking/5gokan_parking_1_img_000170.png",
    "../data/insta_frames/5gokan/5gokan_smoking/5gokan_smoking_0_img_000270.png",
    "../data/insta_frames/5gokan/5gokan_stairs_cnt/5gokan_stairs_cnt_3_img_000010.png",
    "../data/insta_frames/5gokan/5gokan_stairs_west/5gokan_stairs_west_0_img_000340.png",
    "../data/insta_frames/8gokan/8gokan_1F/8gokan_1F_1_img_000000.png",
    "../data/insta_frames/8gokan/8gokan_ent_north/8gokan_ent_north_0_img_000010.png",
    "../data/insta_frames/8gokan/8gokan_ent_south/8gokan_ent_south_1_img_000070.png",
    "../data/insta_frames/denkisogokan/denkisogokan_2F/denkisogokan_2F_2_img_000300.png",
    "../data/insta_frames/denkisogokan/denkisogokan_3F/denkisogokan_3F_0_img_000010.png",
    "../data/insta_frames/denkisogokan/denkisogokan_4F/denkisogokan_4F_0_img_000170.png",
    "../data/insta_frames/denkisogokan/denkisogokan_elevator/denkisogokan_elevator_0_img_000220.png",
    "../data/insta_frames/denkisogokan/denkisogokan_lounge/denkisogokan_lounge_1_img_000060.png",
    "../data/insta_frames/denkisogokan/denkisogokan_stairs/denkisogokan_stairs_0_img_000120.png",
    "../data/insta_frames/lab-lounge/lab-lounge_0_img_000170.png",
    "../data/insta_frames/lab/lab_bs_cnt/lab_bs_cnt_0_img_000270.png",
    "../data/insta_frames/lab/lab_corner/lab_corner_0_img_000270.png",
    "../data/insta_frames/lab/lab_desk/lab_desk_8_img_000210.png",
    "../data/insta_frames/lab/lab_desk_table/lab_desk_table_3_img_000150.png",
    "../data/insta_frames/lab/lab_ent/lab_ent_0_img_000020.png",
    "../data/insta_frames/lab/lab_printer/lab_printer_0_img_000030.png",
    "../data/insta_frames/lab/lab_table/lab_table_0_img_000170.png",
    "../data/insta_frames/lab/lab_wb_cnt/lab_wb_cnt_0_img_000120.png",
    "../data/insta_frames/lab/lab_wb_ent/lab_wb_ent_1_img_000160.png",
    "../data/insta_frames/lounge/lounge_1_img_000060.png",
]

In [10]:
from sklearn.preprocessing import LabelEncoder

In [11]:
le = LabelEncoder() 
le.fit(labs.s_class) 
labs["labels"] = le.transform(labs.s_class) 
labs.groupby('labels')

/home/tamaru/.local/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [12]:
cor_table = labs.groupby('labels').s_class.unique() 
cor_table = pd.DataFrame(cor_table) 
cor_table

,s_class
labels,
0,[lab-lounge]
1,[lab_bs_cnt]
2,[lab_corner]
3,[lab_desk]
4,[lab_desk_table]
5,[lab_ent]
6,[lab_printer]
7,[lab_table]
8,[lab_wb_cnt]


In [13]:
cor_table.to_csv('cor_table.csv', index=False)

In [14]:
dfs = labs.drop(['s_class'], axis=1)
dfs

,img_path,labels
1397,../data/insta_frames/lab-lounge/lab-lounge_0_i...,0
1398,../data/insta_frames/lab-lounge/lab-lounge_0_i...,0
1399,../data/insta_frames/lab-lounge/lab-lounge_0_i...,0
1400,../data/insta_frames/lab-lounge/lab-lounge_0_i...,0
1401,../data/insta_frames/lab-lounge/lab-lounge_0_i...,0
1402,../data/insta_frames/lab-lounge/lab-lounge_0_i...,0
1403,../data/insta_frames/lab-lounge/lab-lounge_0_i...,0
1404,../data/insta_frames/lab-lounge/lab-lounge_0_i...,0
1405,../data/insta_frames/lab-lounge/lab-lounge_0_i...,0
1406,../data/insta_frames/lab-lounge/lab-lounge_0_i...,0


# trainデータ, testデータの分割

In [15]:
from sklearn.model_selection import train_test_split

In [16]:
train_data, test_data = train_test_split(dfs, test_size=0.2, random_state=42, stratify=dfs.labels)

In [17]:
#画像の前処理を定義
data_transforms = {
    'data': transforms.Compose([
        transforms.Resize(224),
        transforms.ToTensor(),
        transforms.Normalize([0.5,0.5,0.5],[0.5,0.5,0.5])
    ])
}
#正規化をしない処理
to_tensor_transforms = transforms.Compose([
    transforms.Resize(224),
    transforms.ToTensor()
])

In [18]:
class CustomDataset(torch.utils.data.Dataset):
        
    def __init__(self, dataframe, root_dir, transform=None):
        #前処理クラスの指定
        self.transform = transform
        #pandasでcsvデータの読み出し
        #画像とラベルの一覧を保持するリスト
        self.images = np.array(dataframe.img_path).tolist()
        self.labels = np.array(dataframe.labels).tolist()
        self.root_dir = root_dir
    def __len__(self):
        return len(self.images)
    
    def __getitem__(self, idx):
        #dataframeから画像へのパスとラベルを読み出す
        label = self.labels[idx]
        img = self.images[idx]
        #画像の読み込み
        with open(img, 'rb') as f:
            image = Image.open(f)
            image = image.convert('RGB')
            image = image.resize((224,224))
        #画像への処理
        if self.transform is not None:
            image = self.transform(image)
            
        return image, label

In [19]:
train_set = CustomDataset(dataframe=train_data, root_dir="../data/insta_frames", transform=data_transforms['data'])
test_set = CustomDataset(dataframe=test_data, root_dir="../data/insta_frames", transform=data_transforms['data'])

In [20]:
# DataLoaderのcollate_fnはバッチ内のtensorのshapeをすべて同じにする必要がある
# 自分で指定してエラーが起きないようにする
def my_collate_fn(batch):
    # datasetの出力が
    # [image, target] = dataset[batch_idx]
    # の場合.
    images = []
    labels = []
    for image, label in batch:
        images.append(image)
        labels.append(label)
    images = torch.stack(images,dim=0)
    return images, labels

In [21]:
train_loader = torch.utils.data.DataLoader(dataset=train_set, batch_size=16, shuffle=True, num_workers=6)
test_loader = torch.utils.data.DataLoader(dataset=test_set, batch_size=16, shuffle=False, num_workers=6)

# ネットワークの定義
vgg16 finetuning

In [22]:
import torchvision.models as models
from s2cnn import SO3Convolution
from s2cnn import S2Convolution
from s2cnn import so3_integrate
from s2cnn import so3_near_identity_grid
from s2cnn import s2_near_identity_grid
import gzip
import pickle
from torch.autograd import Variable
import argparse

In [23]:
num_classes = num_labs_class
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [24]:
class S2ConvNet_deep(nn.Module):

    def __init__(self, bandwidth=30):
        super(S2ConvNet_deep, self).__init__()

        grid_s2    =  s2_near_identity_grid(n_alpha=6, max_beta=np.pi/16, n_beta=1)
        grid_so3_1 = so3_near_identity_grid(n_alpha=6, max_beta=np.pi/16, n_beta=1, max_gamma=2*np.pi, n_gamma=6)
        grid_so3_2 = so3_near_identity_grid(n_alpha=6, max_beta=np.pi/ 8, n_beta=1, max_gamma=2*np.pi, n_gamma=6)
        grid_so3_3 = so3_near_identity_grid(n_alpha=6, max_beta=np.pi/ 4, n_beta=1, max_gamma=2*np.pi, n_gamma=6)
        grid_so3_4 = so3_near_identity_grid(n_alpha=6, max_beta=np.pi/ 2, n_beta=1, max_gamma=2*np.pi, n_gamma=6)

        self.convolutional = nn.Sequential(
            S2Convolution(
                nfeature_in  = 3,
                nfeature_out = 64,
                b_in  = bandwidth,
                b_out = bandwidth,
                grid=grid_s2),
            nn.ReLU(inplace=False),
            SO3Convolution(
                nfeature_in  =  64,
                nfeature_out = 64,
                b_in  = bandwidth,
                b_out = bandwidth//2,
                grid=grid_so3_1),
            nn.ReLU(inplace=False),

            SO3Convolution(
                nfeature_in  = 64,
                nfeature_out = 128,
                b_in  = bandwidth//2,
                b_out = bandwidth//2,
                grid=grid_so3_2),
            nn.ReLU(inplace=False),
            SO3Convolution(
                nfeature_in  = 128,
                nfeature_out = 128,
                b_in  = bandwidth//2,
                b_out = bandwidth//4,
                grid=grid_so3_2),
            nn.ReLU(inplace=False),

            SO3Convolution(
                nfeature_in  = 128,
                nfeature_out = 256,
                b_in  = bandwidth//4,
                b_out = bandwidth//4,
                grid=grid_so3_3),
            nn.ReLU(inplace=False),
            SO3Convolution(
                nfeature_in  = 256,
                nfeature_out = 256,
                b_in  = bandwidth//4,
                b_out = bandwidth//8,
                grid=grid_so3_3),
            nn.ReLU(inplace=False),

            SO3Convolution(
                nfeature_in  = 256,
                nfeature_out = 256,
                b_in  = bandwidth//8,
                b_out = bandwidth//8,
                grid=grid_so3_4),
            nn.ReLU(inplace=False)
            )

        self.linear = nn.Sequential(
            # linear 1
            nn.BatchNorm1d(64),
            nn.Linear(in_features=64,out_features=64),
            nn.ReLU(inplace=False),
            # linear 2
            nn.BatchNorm1d(64),
            nn.Linear(in_features=64, out_features=32),
            nn.ReLU(inplace=False),
            # linear 3
            nn.BatchNorm1d(32),
            nn.Linear(in_features=32, out_features=num_classes)
        )

    def forward(self, x):
        x = self.convolutional(x)
        x = so3_integrate(x)
        x = self.linear(x)
        return x

In [25]:
net = S2ConvNet_deep()
net

S2ConvNet_deep(
  (convolutional): Sequential(
    (0): S2Convolution()
    (1): ReLU()
    (2): SO3Convolution()
    (3): ReLU()
    (4): SO3Convolution()
    (5): ReLU()
    (6): SO3Convolution()
    (7): ReLU()
    (8): SO3Convolution()
    (9): ReLU()
    (10): SO3Convolution()
    (11): ReLU()
    (12): SO3Convolution()
    (13): ReLU()
  )
  (linear): Sequential(
    (0): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (1): Linear(in_features=64, out_features=64, bias=True)
    (2): ReLU()
    (3): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (4): Linear(in_features=64, out_features=32, bias=True)
    (5): ReLU()
    (6): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (7): Linear(in_features=32, out_features=10, bias=True)
  )
)

In [26]:
#パラメータ凍結と採取層クラス数変更
# for param in net.parameters():
#     param.requires_grad = False
#最終層をnum_s_classクラス用に変, lr=args.lr更
# num_ftrs = net.classifier[6].in_features
# net.classifier[6] = nn.Linear(in_features=num_ftrs, out_features=num_classes).to(device)
#最適化関数
criterion = nn.CrossEntropyLoss()
#criterion.to(device)
optimizer = optim.Adam(net.parameters(), lr=5e-3)
net = net.to(device)
net

S2ConvNet_deep(
  (convolutional): Sequential(
    (0): S2Convolution()
    (1): ReLU()
    (2): SO3Convolution()
    (3): ReLU()
    (4): SO3Convolution()
    (5): ReLU()
    (6): SO3Convolution()
    (7): ReLU()
    (8): SO3Convolution()
    (9): ReLU()
    (10): SO3Convolution()
    (11): ReLU()
    (12): SO3Convolution()
    (13): ReLU()
  )
  (linear): Sequential(
    (0): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (1): Linear(in_features=64, out_features=64, bias=True)
    (2): ReLU()
    (3): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (4): Linear(in_features=64, out_features=32, bias=True)
    (5): ReLU()
    (6): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (7): Linear(in_features=32, out_features=10, bias=True)
  )
)

#学習率の変更
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=)

# 学習の実行

In [27]:
#Cross Validationを行いたい...
#Early Stopping を行いたい

num_epochs = 10
train_loss_list = []
train_acc_list = []
val_loss_list = []
val_acc_list = []

for epoch in range(num_epochs):
    train_loss = 0
    train_acc = 0
    val_loss = 0
    val_acc = 0
    
    #train
    net.train()
    for i, (images, labels) in enumerate(train_loader):
        net.train() #?
        images, labels = images.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = net(images)
        loss = criterion(outputs, labels)
        train_loss += loss.item()
        train_acc += (outputs.max(1)[1]==labels).sum().item()
        loss.backward()
        optimizer.step()
        
    avg_train_loss = train_loss/len(train_loader.dataset)
    avg_train_acc = train_acc/len(train_loader.dataset)
    
    #val
    for images, labels in test_loader:
        net.eval()
        with torch.no_grad():
            images = images.to(device)
            labels = labels.to(device)
            outputs = net(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            val_acc += (outputs.max(1)[1]==labels).sum().item()
   
    avg_val_loss = val_loss/len(test_loader.dataset)
    avg_val_acc = val_acc/len(test_loader.dataset)
    
    print('Epoch [{}/{}], Loss: {loss:.4f}, val_loss: {val_loss:.4f}, val_acc: {val_acc:.4f}'.format(epoch+1, num_epochs, i+1, loss=avg_train_loss, val_loss=avg_val_loss, val_acc=avg_val_acc))
    train_loss_list.append(avg_train_loss)
    train_acc_list.append(avg_train_acc)
    val_loss_list.append(avg_val_loss)
    val_acc_list.append(avg_val_acc)

AssertionError: 

# train, validationのloss acc のグラフを作成

In [ ]:
plt.figure()
plt.plot(range(num_epochs), train_loss_list, color='blue', linestyle='-', label='train_loss')
plt.plot(range(num_epochs), val_loss_list, color='green', linestyle='--', label='val_loss')
plt.legend()
plt.xlabel('epoch')
plt.ylabel('loss')
plt.title('Training and validation loss')
plt.grid()

plt.figure()
plt.plot(range(num_epochs), train_acc_list, color='blue', linestyle='-', label='train_acc')
plt.plot(range(num_epochs), val_acc_list, color='green', linestyle='--', label='test_acc')
plt.legend()
plt.xlabel('epoch')
plt.ylabel('acc')
plt.title('Training and validation acc')
plt.grid()

# Grad-CAMの定義

In [ ]:
class GradCam:
    def __init__(self, model):
        self.model = model.eval()
        self.feature = None
        self.gradient = None
    
    def save_gradient(self, grad):
        self.gradient = grad
    
    def __call__(self, x):
        image_size = (x.size(-1), x.size(-2))
        feature_maps =[]
        
        for i in range(x.size(0)):
            img = x[i].data.cpu().numpy() #GPU上のTensorはcpuに移さないとnumpyに変換できない
            img = img - np.min(img)
            if np.max(img) != 0:
                img = img / np.max(img)
            
            feature = x[i].unsqueeze(0)
            
            for name, module in self.model.named_children():
                if name == 'clasifier':
                    feature = feature.view(feature.size(0), -1)
                feature = module(feature)
                if name == 'features':
                    feature.register_hook(self.save_gradient)
                    self.feature = feature
                    
            classes = F.sigmoid(feature)
            one_hot, _ = classes.max(dim=-1)
            self.model.zero_grad()
            one_hot.backward()
            
            weight = self.gradient.mean(dim=-1, keepdim=True).mean(dim=-2, keepdim=True)
            
            mask = F.relu((weight*self.feature).sum(dim=1)).squeeze(0)
            mask = cv2.resize(mask.data.cpu().numpy(), image_size)
            mask = mask - np.min(mask)
            
            if np.max(mask) != 0:
                mask = mask/np.max(mask)
                
            feature_map = np.float32(cv2.applyColorMap(np.uint8(255*mask), cv2.COLORMAP_JET))
            cam = feature_map + np.float32((np.uint8(img.transpose((1,2,0))*225)))
            cam = cam - np.min(cam)
            
            if np.max(cam) != 0:
                cam = cam/np.max(cam)
                
            feature_maps.append(transforms.ToTensor()(cv2.cvtColor(np.uint8(225*cam), cv2.COLOR_BGR2RGB)))
            
        feature_maps = torch.stack(feature_maps)
        
        return feature_maps
                

In [ ]:
for i in range(len(cam_test_img_path)):
    #入力画像の読み込み
    cam_test_img = Image.open(cam_test_img_path[i])
    cam_img_tensor = (data_transforms['data']((cam_test_img))).unsqueeze(dim=0)
    
    cam_img_tensor = cam_img_tensor.to(device)
    
    img_size = cam_test_img.size
    #grad-camによる予測根拠可視化
    gradcam = GradCam(net)
    
    feature_image = gradcam(cam_img_tensor).squeeze(dim=0)
    feature_image = transforms.ToPILImage()(feature_image)
    
    pred_idx = net(cam_img_tensor).max(1)[1]
                      
    save_dir = '../data/gradcam_img/VGG16/'+s_classlist[i]
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)	# Make a directory
    #保存先ディレクトリ名はその画像のクラス，画像の予測値を画像の名前に書き込む
    cv2.imwrite(save_dir+'/heatmap_pred_'+s_classlist[pred_idx]+'.jpg', superimposed_img)
    print('Saved: ', save_dir+'/heatmap_pred_'+s_classlist[pred_idx]+'.jpg')